In [80]:
%reload_ext autoreload
%autoreload 2
#%reload_ext notexbook'
#%texify

#### Generating Evaluation data using 30 random segmentation masks

In [81]:
import numpy as np
import matplotlib.pyplot as plt
import torch

import os

os.chdir('/home/axtr7550/Chromosome_project/cellbgnet/')
import cellbgnet
import cellbgnet.utils

from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.simulation.psf_kernel import SMAPSplineCoefficient
from cellbgnet.generic.emitter import EmitterSet
from cellbgnet.train_loss_infer import generate_probmap_cells

os.chdir('/home/axtr7550/Chromosome_project/cellbgnet/notebooks')

from skimage.io import imread
from skimage.measure import label
import random
import edt
from skimage.filters import gaussian
from scipy.ndimage import rotate
import random
import pickle
import pathlib
from pathlib import Path
%matplotlib qt5

In [82]:
param_file = '../cellbgnet/utils/reference_files/reference.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [83]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
train_size = simulation_params['train_size']

In [84]:
with open(param.Simulation.edt_noise_map_path, 'rb') as fp:
    edt_noise_map = pickle.load(fp)

In [85]:
model = cellbgnet.model.CellBGModel(param)

INITIATING CUDA IMPLEMENTATION
training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21,

#### Read 30 random segmentation masks

In [86]:
filenames = model.data_generator.cell_mask_filenames


In [87]:
random_filenames = random.choices(filenames, k=1)

In [88]:
cell_masks = []
for i, file in enumerate(random_filenames,0):
    img = imread(file)
    #img = rotate(img, random_angles[i], reshape=False)
    cell_masks.append(img)

In [89]:
img.shape

(1041, 1302)

In [90]:
cell_masks_batch = np.stack(cell_masks)

In [91]:
cell_masks_batch.shape

(1, 1041, 1302)

In [92]:
plt.figure()
plt.imshow(cell_masks_batch[0])
plt.show()

In [93]:
prob_map = (cell_masks_batch > 0) * 0.003125

In [94]:
plt.figure()
plt.imshow(prob_map[0])
plt.show()

In [95]:
data_gen = DataSimulator(psf_params, simulation_params, hardware_params)

INITIATING CUDA IMPLEMENTATION


In [96]:
S, X_os, Y_os, Z, I, field_xy = data_gen.sampling(batch_size=1, prob_map=gpu(prob_map), local_context=None,
                                                  iter_num=None)

In [97]:
S.shape

torch.Size([1, 1, 1041, 1302])

In [98]:
plt.figure()
plt.imshow(S[0][0].cpu().numpy())
plt.show()

In [99]:
index = 0
fig, ax = plt.subplots(nrows=1, ncols=6)
ax[0].imshow(S[index][0].cpu().numpy())
ax[0].set_title('Dot location')
x_os_img = ax[1].imshow(X_os[index][0].cpu().numpy())
ax[1].set_title('X offset')
#fig.colorbar(x_os_img, ax=ax[1])
y_os_img = ax[2].imshow(Y_os[index][0].cpu().numpy())
ax[2].set_title('Y offset')
#fig.colorbar(y_os_img, ax=ax[2])
z_vals_img = ax[3].imshow(Z[index][0].cpu().numpy())
ax[3].set_title('Z value')
#fig.colorbar(z_vals_img, ax=ax[3])
i_vals_img = ax[4].imshow(I[index][0].cpu().numpy())
ax[4].set_title('Intensity Value')
#fig.colorbar(i_vals_img, ax=ax[4])
ax[5].imshow(cell_masks_batch[index])
ax[5].set_title('cell mask')
plt.show()

In [100]:
imgs_sim = data_gen.simulate_psfs(S, X_os, Y_os, Z, I)
#6 seconds on cuda for 128

In [101]:
#imgs_sim.to('cpu');
#torch.cuda.empty_cache()

In [103]:
index = 0
fig, ax = plt.subplots(nrows=1, ncols=5)
ax[0].imshow(prob_map[index])
ax[0].set_title('Probabilty map')
ax[1].imshow(cell_masks_batch[index])
ax[1].set_title('Cell mask sampled')
ax[2].imshow(imgs_sim[index, 0].cpu().numpy())
ax[2].set_title('Simulated PSFs')
ax[3].imshow(S[index, 0].cpu().numpy())
ax[3].set_title('Emitter pixel locs')
ax[4].imshow(I[index, 0].cpu().numpy() * psf_params['photon_scale'])
ax[4].set_title('Intensity values')
plt.show()

In [104]:
S.shape

torch.Size([1, 1, 1041, 1302])

In [105]:
xyzi = torch.cat([X_os[:, :, None], Y_os[:, :, None], Z[:, :, None], I[:, :, None]], 2)
xyzi = xyzi[:, 0]

In [106]:
S = S[:, 0]

In [107]:
s_inds = tuple(S.nonzero().transpose(1, 0))
# get these molecules' sub-pixel xy offsets, z positions and photons
xyzi_true = xyzi[s_inds[0], :, s_inds[1], s_inds[2]]

In [108]:
xyzi_true

tensor([[ 0.0616, -0.1612,  0.8130,  0.2678],
        [-0.0990,  0.3908, -0.7039,  0.7200],
        [-0.3590,  0.2523,  0.8704,  0.5014],
        ...,
        [-0.0257, -0.2856, -0.0415,  0.9694],
        [ 0.1294, -0.1374, -0.8576,  0.6913],
        [-0.1233,  0.3096,  0.3633,  0.5354]], device='cuda:0')

In [109]:
xyzi_true[:, 0] += s_inds[2] + 0.5

In [110]:
xyzi_true[:, 1] += s_inds[1] + 0.5

In [111]:
pos_tar = xyzi_true[:, :3]

In [112]:
pos_tar.shape

torch.Size([353, 3])

In [113]:
pos_tar[:, 2] *= 500

In [114]:
pos_tar

tensor([[ 565.5616,   99.3388,  406.5049],
        [ 780.4010,   99.8908, -351.9264],
        [1039.1410,  100.7523,  435.1767],
        ...,
        [ 558.4743,  998.2144,  -20.7447],
        [ 556.6295, 1000.3625, -428.8164],
        [ 528.3767, 1001.8096,  181.6270]], device='cuda:0')

In [115]:
xyzi_true[0, 3]

tensor(0.2678, device='cuda:0')

In [117]:
plt.figure()
plt.imshow((single_psfs[0] * 0.3924 *  3000).numpy())
plt.show()

type: name 'single_psfs' is not defined

In [118]:
from cellbgnet.utils.plot_funcs import PlotFrameCoord

In [119]:
imgs_sim.shape

torch.Size([1, 1, 1041, 1302])

In [120]:
plt.figure()
PlotFrameCoord(imgs_sim[0].cpu(), pos_tar=pos_tar.cpu(),plot_colorbar_frame=True, annotate_tar_z=True).plot()
plt.show()

In [121]:
plt.figure()
plt.imshow(imgs_sim[13, 0].cpu().numpy())
plt.show()

type: index 13 is out of bounds for dimension 0 with size 1

### Noise adding inside and outside cells

In [122]:

dists = np.zeros_like(cell_masks_batch)
for i in range(len(cell_masks_batch)):
    dists[i] = edt.edt(cell_masks_batch[i])

In [123]:
dists.shape

(1, 1041, 1302)

In [125]:
index = 0
fig, ax = plt.subplots(nrows=1, ncols=3)
ax[0].imshow(cell_masks_batch[index])
ax[0].set_title('Cell mask')
ax[1].imshow(dists[index])
ax[1].set_title('Edt')
ax[2].imshow(prob_map[index])
ax[2].set_title('Prob map')
plt.show()

In [126]:
def mean_bg_mask(dists, mean_map):
    dists_copy = np.copy(dists)
    for edt_val, mean_bg in mean_map.items():
        dists_copy[dists == int(edt_val)] = mean_bg
    return dists_copy

In [127]:
def variance_bg_mask(dists, variance_map):
    dists_copy = np.copy(dists)
    for edt_val, stddev_bg in variance_map.items():
        dists_copy[dists == int(edt_val)] = stddev_bg
    return dists_copy

In [128]:
dists = np.ceil(dists)

In [129]:
#mean_bg_cells = mean_bg_mask(dists, mean_map)
#stddev_bg_cells = variance_bg_mask(dists, stddev_map)
fitted_beta_map = edt_noise_map['betas']
fitted_alpha_map = edt_noise_map['alphas']
alpha_bg_cells = mean_bg_mask(dists, fitted_alpha_map)
beta_bg_cells = variance_bg_mask(dists, fitted_beta_map)

In [131]:
index = 0
fig, ax = plt.subplots(nrows=1, ncols=2)
#ax[0].imshow(mean_bg_cells[index])
ax[0].imshow(alpha_bg_cells[index])
ax[0].set_title('Alpha values')
#ax[0].set_title('Mean bg values')
#ax[1].imshow(stddev_bg_cells[index])
ax[1].imshow(beta_bg_cells[index])
#ax[1].set_title('Std dev bg values')
ax[1].set_title('Beta values')
plt.show()

In [132]:
alpha_t = torch.from_numpy(alpha_bg_cells)
beta_t = 1.0/torch.from_numpy(beta_bg_cells)

In [133]:
alpha_t.shape, beta_t.shape

(torch.Size([1, 1041, 1302]), torch.Size([1, 1041, 1302]))

In [134]:
m = torch.distributions.gamma.Gamma(concentration=alpha_t, rate=beta_t)
sample = m.sample()

In [136]:
index = 0
fig, ax = plt.subplots(nrows=1, ncols=3)
#ax[0].imshow(mean_bg_cells[index])
ax[0].imshow(alpha_bg_cells[index])
ax[0].set_title('Alpha bg cells')
#ax[0].set_title('Mean bg values')
#ax[1].imshow(stddev_bg_cells[index])
#ax[1].set_title('Std dev bg values')
ax[1].imshow(beta_bg_cells[index])
ax[1].set_title('Beta bg cells')
img = ax[2].imshow(sample[index].cpu().numpy(), cmap='gray')
ax[2].set_title('Sampled bg values')
fig.colorbar(img, ax=ax[2])
plt.show()

In [138]:
index = 0
plt.figure()
plt.imshow(sample[index].cpu().numpy(), cmap='gray')
plt.show()

### Dot overlay

In [139]:
baseline = 103.0
e_per_adu = 0.39
qe = 0.95
bg_photons = (sample - baseline) * e_per_adu / qe

In [140]:
bg_photons = bg_photons[:, None]

In [141]:
bg_photons = torch.clamp(bg_photons, min=0.0)

In [142]:
imgs_sim.shape, bg_photons.shape

(torch.Size([1, 1, 1041, 1302]), torch.Size([1, 1, 1041, 1302]))

In [143]:
bg_photons = bg_photons.to('cuda:0')

In [144]:
plt.figure()
plt.imshow(bg_photons[index, 0].cpu().numpy())
plt.show()

In [145]:
after_dots = imgs_sim + bg_photons


In [149]:
S.shape

torch.Size([1, 1041, 1302])

In [150]:
index = 0
fig, ax = plt.subplots(nrows=1, ncols=6)
ax[0].imshow(prob_map[index])
ax[0].set_title('Probabilty map')
ax[1].imshow(cell_masks_batch[index])
ax[1].set_title('Cell mask sampled')
ax[2].imshow(imgs_sim[index, 0].cpu().numpy())
ax[2].set_title('Simulated PSFs')
#ax[3].imshow(S[index, 0].cpu().numpy())
ax[3].set_title('Emitter pixel locs')
ax[4].imshow(I[index, 0].cpu().numpy() * psf_params['photon_scale'])
ax[4].set_title('Intensity values')
ax[5].imshow(after_dots[index][0].cpu().numpy(), cmap='gray')
ax[5].set_title('After overlaying dots')
plt.show()

### Adding camera noise

In [151]:
RN = 2.2

In [152]:
after_dots = torch.distributions.Poisson(after_dots * qe).sample()

In [153]:
zeros = torch.zeros_like(after_dots)

In [154]:
readout_noise = torch.distributions.Normal(zeros, zeros + RN).sample()

In [155]:
after_dots = after_dots + readout_noise

In [156]:
final_dots_for_net = torch.clamp((after_dots/ e_per_adu) + baseline, min=0)

In [161]:
index = 0
fig, ax = plt.subplots(nrows=1, ncols=6)
ax[0].imshow(sample[index].cpu().numpy(), cmap='gray')
ax[0].set_title('Sampled background ADU')
ax[1].imshow(cell_masks_batch[index])
ax[1].set_title('Cell mask sampled')
ax[2].imshow(imgs_sim[index, 0].cpu().numpy())
ax[2].set_title('Simulated PSFs')
#ax[3].imshow(S[index, 0].cpu().numpy())
#ax[3].set_title('Emitter pixel locs')
ax[3].imshow(I[index, 0].cpu().numpy() * psf_params['photon_scale'])
ax[3].set_title('Intensity values')
ax[4].imshow(after_dots[index][0].cpu().numpy(), cmap='gray')
ax[4].set_title('After overlaying dots (photons)')
ax[5].imshow(final_dots_for_net[index][0].cpu().numpy(), cmap='gray')
ax[5].set_title('After camer noise (ADU)')
plt.tight_layout()
plt.show()

In [162]:
plt.figure()
plt.imshow(final_dots_for_net[index][0].cpu().numpy(), cmap='gray')
plt.show()

In [ ]:
plt.figure()
plt.imshow()

In [ ]:
index = 3
plt.figure()
plt.imshow(final_dots_for_net[index][0].cpu().numpy(), cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
final_dots_for_net[index][0].cpu().numpy().max()

331.9748

In [ ]:
final_dots_for_net[index][0].cpu().numpy().min()

76.83658